# Example HLS inference

This notebook demonstrates an example of inferencing on the fine-tuned HLS Foundation Model using HLS data in the cloud and allows users to explore and select regions of interest.


### Setup
- Change kernel to `prithvi_eo`

## Deployment to Sagemaker Endpoint

In [ ]:
import boto3
import yaml
from sagemaker import get_execution_role


In [ ]:
# Since we are downloading data from the internet, platform is used. Else, VPC is prefered.
image_config = {
     'RepositoryAccessMode': 'Platform'
}
ACCOUNT_ID = boto3.client('sts').get_caller_identity().get('Account')
IMAGE_URI = f'{ACCOUNT_ID}.dkr.ecr.us-west-2.amazonaws.com/prithvi_inference'


In [ ]:
model_identifier = "Prithvi-EO"
usecase = 'burn_scars' # define usecase
# read from file

with open(f'../../{model_identifier}/variables.yaml') as variable_file:
    variables = yaml.safe_load(variable_file)

identifier = variables['identifier']
model_name = variables['model_name']
config_filename = variables['config_filename']
BUCKET_NAME = variables['bucket_name']


# environment variables for inference endpoint
ENV = {
    "CHECKPOINT_FILENAME": "s3://hdcrs-school-2025-06031643-tvsscp-001/models/prithvi_eo_burn_scars.ckpt",
    "S3_CONFIG_FILENAME": "s3://hdcrs-school-2025-06031643-tvsscp-001/configs/prithvi_eo_burn_scars-config.yaml",
    "BUCKET_NAME": "hdcrs-school-2025-06031643-tvsscp-001",
    "USECASE": usecase,
    "SAGEMAKER_CONTAINER_LOG_LEVEL": "20"
}

# Configuration for primary container where the model will be deployed
primary_container = {
    'ContainerHostname': 'ModelContainer',
    'Image': IMAGE_URI,
    'ImageConfig': image_config,
    'Environment': ENV
}

In [ ]:
# Create Sagemaker model holder
model_name = f'{identifier}-{model_identifier}'
execution_role_arn = get_execution_role()


sagem = boto3.client('sagemaker')

# Create model based on custom code and artifacts in sagemaker
resp = sagem.create_model(
        ModelName=model_name,
        ExecutionRoleArn=execution_role_arn,
        PrimaryContainer=primary_container,
    )


In [ ]:
# Create Sagemaker endpoint configuration using the model that was created
endpoint_config_name = f'{model_name}-endpoint-config'

# Create endpoint config for easier deployment
sagem.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            'VariantName': 'V1',
            'ModelName': model_name,
            'InitialInstanceCount': 1,
            'InitialVariantWeight': 1,
            'InstanceType': 'ml.p3.2xlarge'
        },
    ],
)


In [ ]:
# Create Sagemaker endpoint using the endpoint configuration that was created
endpoint_name = f'{model_name}-endpoint'

# Create endpoint i.e Deployment.
sagem.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name,
)

resp = sagem.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Endpoint Status: " + status)

print('Waiting for {} endpoint to be in service...'.format(endpoint_name))
waiter = sagem.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

# Use deployed Prithvi model

We need to install the leafmap client for data visualization in the notebook.

![Prithvi Inference](../../images/HLS-inference.png)

Import the python libraries required for running the script

In [ ]:
import json
import ipyleaflet
import numpy as np

This is a dictionary contains several sample cases of burn scars across the globe. These configuration settings will automatically set the location of the map for AOI selection and the start_date and end_date to perform the data query.

In [ ]:
#configure settings for selected events

EVENT_DETAILS = {
    'mongolian_fire': {
        'center_lon': 119.3,
        'center_lat': 47.1,
        'default_zoom': 8,
        'start_date': '2022-04-19T00:00:00Z',
        'end_date': '2022-04-19T23:59:59Z'
    },
    'new_mexico_black_fire': {
        'center_lon': -107.5,
        'center_lat': 33.5,
        'default_zoom': 10,
        'start_date': '2022-05-16T00:00:00Z',
        'end_date': '2022-06-10T23:59:59Z'
    },
    'alberta_fire': {
        'center_lon': -124.2,
        'center_lat': 61.8,
        'default_zoom': 8,
        'start_date': '2023-05-27T00:00:00Z',
        'end_date': '2023-05-28T23:59:59Z'
    },
    "arizona_fire": {
        'start_date': '2024-10-06T00:00:00Z',
        'end_date': '2024-10-06T23:59:59Z'
        "center_lon": -112.0036,
        "center_lat": 33.9553,
        'default_zoom': 10
    }
}

Select the predefined event of your choice from above. If you'd like to execute a different event, the simplest way to implement it would be to add the event details as a new key in the dictionary

In [ ]:
event = 'maui_fire'
event_details = EVENT_DETAILS[event]

In [ ]:
datestring = event_details['start_date']
HLSL30_TILE_LAYER = 'https://gitc-a.earthdata.nasa.gov/wmts/epsg3857/best/wmts.cgi?TIME=' + datestring + '&layer=HLS_L30_Nadir_BRDF_Adjusted_Reflectance&style=default&tilematrixset=GoogleMapsCompatible_Level12&Service=WMTS&Request=GetTile&Version=1.0.0&Format=image%2Fpng&TileMatrix={z}&TileCol={x}&TileRow={y}'
HLSS30_TILE_LAYER = 'https://gitc-a.earthdata.nasa.gov/wmts/epsg3857/best/wmts.cgi?TIME=' + datestring + '&layer=HLS_S30_Nadir_BRDF_Adjusted_Reflectance&style=default&tilematrixset=GoogleMapsCompatible_Level12&Service=WMTS&Request=GetTile&Version=1.0.0&Format=image%2Fpng&TileMatrix={z}&TileCol={x}&TileRow={y}'

Initialize map with one of the canned examples listed above, and HLS XYZ Layers from NASA Worldview. If you would like execute a custom use case, please navigate to your desired AOI, draw a bounding box, and update the start_date and end_date in the prepare_items method

In [ ]:
from ipyleaflet import Map, TileLayer, DrawControl, GeoJSON

In [ ]:
layer = TileLayer(url=HLSL30_TILE_LAYER, attribution='NASA',name='HLSL30', opacity=1)
draw_control = DrawControl()
map = Map(
        default_tiles=layer,
        center=(event_details['center_lat'],
        event_details['center_lon']),
        zoom=event_details['default_zoom']
    )

draw_control.rectangle = {
    "shapeOptions": {
        "fillColor": "#fca45d",
        "color": "#fca45d",
        "fillOpacity": 0.3
    }
}
hlsl30_tile_layer = TileLayer(url=HLSL30_TILE_LAYER, name='HLSL30', attribution='NASA')
hlss30_tile_layer = TileLayer(url=HLSS30_TILE_LAYER, name='HLSL30', attribution='NASA')
map.add_layer(hlsl30_tile_layer)
map.add_layer(hlss30_tile_layer)

map.add(draw_control)

# Store drawn shapes
drawn_shapes = []

# Define a function to handle drawing events
def handle_draw(self, action, geo_json):
    if action == 'created':
        drawn_shapes.append(geo_json)
        print("Shape added.")

# Attach the drawing event handler to the drawing control
draw_control.on_draw(handle_draw)
# map.add(layer)
map

This cell saves the bounding box drawn by the user.

**WARNING:** if you skip the next step, the notebook will not find any HLS data as there will be no bounding box information.

**WARNING:** if you skipped the previous step, the notebook will not find any HLS data as there will be no bounding box information.

In [ ]:
# Read the bounding box and print
bbox = drawn_shapes[0]

We will utilize the trained model, changed configuration file, and the date information to get a prediction on the selected region.

In [ ]:

def bbox_from_geojson(bbox):
    """
        Get the coordinates of bounding box from an extended notation to flat coordinate
        notation
    Args:
        geojson: File path of geojson

    Returns:
        list: [left, down, right, top]
    """
    coordinates = np.asarray(bbox['geometry']['coordinates'])
    lats = coordinates[:, :, 1]
    lons = coordinates[:, :, 0]
    return [lons.min(), lats.min(), lons.max(), lats.max()]

In [ ]:
# Convert geojson to flat bounding box representation.
bbox = bbox_from_geojson(bbox)
bbox

Note: `BUCKET_NAME`, and `identifier` are variables set in the previous notebook. Please copy paste those variables here for this step to run smoothly.


In [ ]:
!pip install boto3

In [ ]:
import sagemaker
import json

# prepare payload
sm = sagemaker.Session().sagemaker_runtime_client
import json

query = {
    'bounding_box': bbox,
    'date': datestring.split('T')[0],
    'model_id': 'burn'
}

response = sm.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(query),
    ContentType="application/json"
)

predictions = json.loads(response['Body'].read())


In [ ]:
geojson

In [ ]:
geojson = predictions['burn_scars']['predictions']

detection_map = Map(
        center=(event_details['center_lat'],
        event_details['center_lon']),
        zoom=event_details['default_zoom'],
    )
detection_map.add(hlsl30_tile_layer)
detection_map.add(hlss30_tile_layer)
detection_map.add(GeoJSON(data=geojson))

detection_map

# For Terramind inference

In [ ]:
# Helper function for plotting both modalities
import numpy as np

import matplotlib.pyplot as plt
import numpy as np
import os
import rasterio

def plot_sample(sample):
    s1 = sample['image']['S1GRD']
    s2 = sample['image']['S2L1C']
    mask = sample['mask']

    # Scaling data. Using -30 to 0 scaling for S-1 and 0 - 2000 for S-2. S-1 is visualized as [VH, VV, VH]
    s1 = (s1.clip(-30, 0) / 30 + 1) * 255
    s2 = (s2.clip(0, 2000) / 2000) * 255
    s1_rgb = np.stack([s1[1], s1[0], s1[1]], axis=0).astype(np.uint8).transpose(1,2,0)
    s2_rgb = s2[[3,2,1]].astype(np.uint8).transpose(1,2,0)

    fig, ax = plt.subplots(1, 3, figsize=(12, 4))
    ax[0].imshow(s1_rgb)
    ax[0].set_title('S-1 GRD')
    ax[0].axis('off')
    ax[1].imshow(s2_rgb)
    ax[1].set_title('S-2 GRD')
    ax[1].axis('off')
    ax[2].imshow(mask, vmin=-1, vmax=1, interpolation='nearest')
    ax[2].set_title('Mask')
    ax[2].axis('off')
    fig.tight_layout()
    plt.show()

In [ ]:
# Use Deployed Terramind model

import sagemaker
import json
import rasterio

# prepare payload
sm = sagemaker.Session().sagemaker_runtime_client

# plot some samples
import random
from glob import glob

samples = glob("../../Terramind/data/sen1floods11_v1.1/data/S2*/*.tif")
samples = random.sample(samples, 3)

for sample in samples:
    s3_sample = sample.replace('../../Terramind/data/sen1floods11_v1.1/', f's3://{BUCKET_NAME}/data/')
    print(sample)
    s1_file = sample.replace('S2L1CHand', 'S1GRDHand').replace('S2Hand', 'S1Hand')
    s2_file = sample.replace('S1GRDHand', 'S2L1CHand').replace('S1Hand', 'S2Hand')
    query = {
        'terramind': True,
        'file_urls': [s3_sample],
        'model_id': usecase
    }

    response = sm.invoke_endpoint(
        EndpointName=endpoint_name,
        Body=json.dumps(query),
        ContentType="application/json"
    )
    predictions = json.loads(response['Body'].read())
    mask_file = predictions[usecase]['s3_link']
    updated_values = {
        'image': {
            'S1GRD': rasterio.open(s1_file).read(),
            'S2L1C': rasterio.open(s2_file).read(),
        },
        'mask': rasterio.open(mask_file).read()[0]
    }
    plot_sample(updated_values)


In [ ]:
# Cleanup

sagem.delete_endpoint(EndpointName=endpoint_name)
sagem.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sagem.delete_model(ModelName='jj-Terramind')